In [26]:
import requests
from bs4 import BeautifulSoup

from instagram_private_api import Client, ClientCompatPatch
from PIL import Image
from io import BytesIO

import numpy as np
import scipy
import scipy.cluster
import binascii

In [27]:
# pull insta page
page = requests.get("https://www.instagram.com/mkchroma/")
soup = BeautifulSoup(page.content, 'html.parser')


In [28]:
# log in
user_name = 'mkchroma'

In [29]:
# query Instagram
api = Client(user_name, password)
results = api.self_feed()
items = [item for item in results.get('feed_items', [])
         if item.get('media_or_ad')]
for item in items:
    # Manually patch the entity to match the public api as closely as possible, optional
    # To automatically patch entities, initialise the Client with auto_patch=True
    ClientCompatPatch.media(item['media_or_ad'])
    print(item['media_or_ad']['code'])

In [35]:
def do_color_things(im):
    NUM_CLUSTERS = 10

    im = im.resize((im.width//5, im.height//5))      # optional, to reduce time
    ar = np.asarray(im)
    shape = ar.shape
    ar = ar.reshape(np.product(shape[:2]), shape[2]).astype(float)

    codes, dist = scipy.cluster.vq.kmeans(ar, NUM_CLUSTERS)

    vecs, dist = scipy.cluster.vq.vq(ar, codes)         # assign codes
    counts, bins = np.histogram(vecs, len(codes))    # count occurrences

    index_max = np.argmax(counts)                    # find most frequent
    peak = codes[index_max]
    color = binascii.hexlify(str.encode(''.join(chr(int(c)) for c in peak)))
    return peak, color

In [37]:
count = 0
for x in results['items']:
    count += 1
    if 'carousel_media' in x.keys():
        img_url = x['carousel_media'][0]['image_versions2']['candidates'][0]['url']
    else:
        img_url = x['image_versions2']['candidates'][0]['url']
        
    response = requests.get(img_url)
    im = Image.open(BytesIO(response.content))
    peak, color = do_color_things(im)
    test_img = Image.new('RGB', (im.width, im.height),tuple([int(x) for x in peak]))
    test_img.show()
    #print('{}: {} {}'.format(count, peak, color))

In [ ]:
#hex_code = [x for x in colors[0]][0]

In [ ]:
test_img = Image.new('RGB', (im.width, im.height),tuple([int(x) for x in peak]))

In [ ]:
test_img.show()

In [ ]:
im.show()

In [ ]:
# https://stackoverflow.com/questions/3241929/python-find-dominant-most-common-color-in-an-image
# found link twice